In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import time

from sqlalchemy import create_engine

## Energy potential: Bhumi (read pdf and figure out terms) & Ryan Cheng (Cleaning the data) US_Renewable_Energy_Technical_Potential

In [ ]:
# opening the file and creating a df
potential_energy_path = 'Resources/Raw Data/US_Renewable_Energy_Technical_Potential.csv'
potential_df = pd.read_csv(potential_energy_path)

In [ ]:
# dropping the unnessecary columns of the df
potential_df = potential_df.drop({'urbanUtilityScalePV_GW', 'urbanUtilityScalePV_km2', 
                                'ruralUtilityScalePV_GW', 'ruralUtilityScalePV_km2', 'rooftopPV_GW', 
                                'CSP_GW', 'CSP_km2', 'onshoreWind_GW', 'onshoreWind_km2', 'offshoreWind_GW', 
                                'offshoreWind_km2', 'biopowerSolid_GW', 'biopowerSolid_BDT', 
                                'biopowerGaseous_GW', 'biopowerGaseous_Tonnes-CH4', 
                                'geothermalHydrothermal_GW', 'EGSGeothermal_GW', 'hydropower_GW', 
                                'hydropower_countOfSites'}, axis = 1)
# naming the 1st column as it was not named before
potential_df.rename(columns = {'Unnamed: 0':'state'}, inplace=True)
potential_df1 = potential_df.fillna(0)

In [ ]:
# creating new columns for each type of energy (solar, wind, bio, geothermal, hydro) by summing the
# respective data
solar_list = ['urbanUtilityScalePV_GWh', 'ruralUtilityScalePV_GWh', 'rooftopPV_GWh', 
                                    'CSP_GWh']
potential_df1['solar_potential_gwh'] = potential_df1[solar_list].sum(axis=1)

wind_list = ['onshoreWind_GWh', 'offshoreWind_GWh']
potential_df1['wind_potential_gwh'] = potential_df1[wind_list].sum(axis=1)

bio_list = ['biopowerSolid_GWh', 'biopowerGaseous_GWh']
potential_df1['biopower_potential_gwh'] = potential_df1[bio_list].sum(axis=1)

geo_list = ['geothermalHydrothermal_GWh', 'EGSGeothermal_GWh']
potential_df1['geothermal_potential_gwh'] = potential_df1[geo_list].sum(axis=1)

potential_df1['hydropower_potential_gwh'] = potential_df1['hydropower_GWh']

# select columns
potential_df1 = potential_df1[['state','solar_potential_gwh','wind_potential_gwh','biopower_potential_gwh','geothermal_potential_gwh','hydropower_potential_gwh']]

# # Save the dataframe as csv
# potential_df1.to_csv("Resources/Clean Data/US_Renewable_Energy_Technical_Potential_Formated.csv",index=False)


## Webscraping

In [ ]:
# proving url address
url  = "https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population#cite_note-5"

# sleep for 1 second before reading url
time.sleep(1)

# scrape wiki url
wiki_data = pd.read_html(url)
# type(wiki_data)

# select the first table from the list of tables and convert to DataFrame
population_table = pd.DataFrame(wiki_data[0])

# dropping first level of multi index column headers
population_2021 = population_table.droplevel(0, axis=1) 

# selecting state and population as of 2021 columns
population_2021 = population_2021 [['State or territory','July 1, 2021']]

# rename columns 
population_2021 = population_2021.rename(columns=({'State or territory':'state','July 1, 2021':'population'}))

# removing extra rows
population_2021 = population_2021[0:52]

# remving territories
population_2021 = population_2021.drop(labels=[29], axis=0)

# # Save the dataframe as csv
# population_2021.to_csv("Resources/Clean Data/US_States_Population_Formated.csv",index=False)


## Energy_consumption: Rafael (TBD) & Ryan Callaghan (TBD) US_Total_Energy_Consumption

In [ ]:
# opening the file and creating a df
energy_consumption_path = 'Resources/Raw Data/US_Total_Energy_Consumption.csv'
energy_consumption_df = pd.read_csv(energy_consumption_path)

In [ ]:
# Change state abb. to names

dict = {
    "AK": "Alaska", "AL":"Alabama", "AR": "Arkansas", "AZ": "Arizona", "CA": "California","CO": "Colorado", "CT": "Connecticut",
"DC": "District of Columbia","DE":"Delaware","FL":"Florida","GA": "Georgia",
"HI":"Hawaii","IA": "Iowa","ID":"Idaho","IL": "Illinois","IN":"Indiana",
"KS": "Kansas","KY" :"Kentucky","LA": "Louisiana","MA":"Massachusetts","MD":"Maryland",
"ME" :"Maine","MI":"Michigan","MN":"Minnesota","MO": "Missouri","MS" : "Mississippi","MT":"Montana",
"NC": "North Carolina","ND": "North Dakota","NE":"Nebraska","NH":"New Hampshire","NJ":"New Jersey","NM": "New Mexico",
"NV": "Nevada","NY": "New York","OH": "Ohio","OK": "Oklahoma","OR": "Oregon","PA":"Pennsylvania",
"RI":"Rhode Island","SC": "South Carolina","SD":"South Dakota","TN": "Tennessee","TX": "Texas",
"UT": "Utah","VA": "Virginia","VT":"Vermont","WA":"Washington","WI":"Wisconsin",
"WV": "West Virginia","WY":"Wyoming"
}

# mapping the state names
energy_consumption_df["State"] = energy_consumption_df["State"].map(dict)

#selecting 
energy_consumption_df = energy_consumption_df[['State','Total Energy Consumed per Capita, million Btu']]

# renaming columns
energy_consumption_df.rename(columns = {'State':'state',
                                        'Total Energy Consumed per Capita, million Btu':'total_energy_consumed'}, 
                                         inplace=True)

In [ ]:
#joining populatioin table and energy consumption table
energy_consumption_df = energy_consumption_df.merge(population_2021, on='state',how="inner")

#calculating total consumption and converting BTU's to Gigawatt hours
consumption_gwh = round((energy_consumption_df['total_energy_consumed'] *  0.00000000029 * 1000000  \
                                        * energy_consumption_df['population']))

#adding calculated values back to the dataframe
energy_consumption_df['total_energy_consumed_gwh'] = consumption_gwh
energy_consumption_df = energy_consumption_df[['state','total_energy_consumed_gwh']]

# # Save the dataframe as csv
# energy_consumption_df.to_csv("Resources/Clean Data/US_Total_Energy_Consumption_Formated.csv",index=False)


In [ ]:
# creating a master table for sql ingestion, plus merging tables

potential_energy_total = potential_df1.set_index('state')
potential_energy_total = pd.DataFrame(potential_energy_total.sum(axis=1))
potential_energy_total = potential_energy_total.rename(columns={0:'total_energy_potential_gwh'})

#merging a set of tables calculated above
master_energy = energy_consumption_df.merge(potential_energy_total, on='state')

#calculating energy excess capacity
energy_excess_capacity = master_energy['total_energy_potential_gwh']-master_energy['total_energy_consumed_gwh']

# adding calculation back to dataframe
master_energy['energy_excess_capacity'] = energy_excess_capacity

# merging the final table with population table and energy types
master_energy = ((master_energy.merge(potential_df1,on='state')).merge(population_2021,on='state'))

# # Save the dataframe as csv
# master_energy.to_csv("Resources/Clean Data/US_Master_Energy.csv",index=False)

In [ ]:
# create database connection
connection_string = "postgres:postgres@localhost:5432/energy_db"
engine = create_engine(f'postgresql://{connection_string}')

# Confirm tables
engine.table_names()

#Loading dataframe into sql database table
master_energy.to_sql(name='energy', con=engine, if_exists='append', index=False)